In [2]:
%load_ext watermark
%watermark -a Chan -d -v -p numpy,pandas,konlpy,tensorflow

Chan 2019-08-07 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
tensorflow 1.13.1


In [14]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras
from tqdm import tqdm
from collections import Counter
import pickle

In [4]:
# 각 동영상별로 count해야한다. 일단 dump파일로 짜놓기

In [5]:
dump_data = pd.read_csv("../../data/sample (7).csv", engine='python',names=['chat', 'label'])

In [6]:
dump_data.head()

,chat,label
0,"님, 50000원 후원 그라샤! - 브베야 오늘 이것만 주고 간다. 그리고 춤좀 그...",0
1,어켜졌당,0
2,브베형,0
3,아니,0
4,싹둑이 채팅창 관리 중입니다,0


In [9]:
len(dump_datap_data)

500

In [11]:
morp = Okt()
morped = [morp.pos(_, norm=True, join=True) for _ in tqdm(dump_data.chat)] # Stemming은 하지 않는다.

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 35.65it/s]

In [12]:
morped

[['님/Suffix',
  ',/Punctuation',
  '50000원/Number',
  '후원/Noun',
  '그라샤/Noun',
  '!/Punctuation',
  '-/Punctuation',
  '브베/Noun',
  '야/Josa',
  '오늘/Noun',
  '이/Determiner',
  '것/Noun',
  '만/Josa',
  '주고/Verb',
  '간다/Noun',
  './Punctuation',
  '그리고/Conjunction',
  '춤좀/Noun',
  '그만/Adverb',
  '춰/Verb',
  '땅/Noun',
  '꺼지겠어/Verb'],
 ['어/Noun', '켜졌당/Verb'],
 ['브베/Noun', '형/Suffix'],
 ['아니/Adjective'],
 ['싹둑/Noun',
  '이/Josa',
  '채팅/Noun',
  '창/Noun',
  '관리/Noun',
  '중/Noun',
  '입니다/Adjective'],
 ['왜/Noun', '또/Noun', '늦었샤/Noun', 'ㅡㅡ/KoreanParticle'],
 ['?!????/Punctuation', '???/Punctuation'],
 ['왜캐늦게왓샤/Noun'],
 ['싹둑/Noun',
  '이/Josa',
  '채팅/Noun',
  '창/Noun',
  '관리/Noun',
  '중/Noun',
  '입니다/Adjective'],
 ['야/Exclamation',
  '브베/Noun',
  '야/Josa',
  '너/Noun',
  '난리/Noun',
  '났더라/Verb',
  '어제/Noun',
  '일/Modifier',
  '때문/Noun',
  '에/Josa',
  '기자/Noun',
  '들/Suffix',
  '이/Josa',
  '너/Noun',
  '찾는대/Verb',
  '~/Punctuation'],
 ['브베/Noun', '미애/Noun', '장례식/Noun', '치르고/Verb', '왔노/Verb'],
 ['케/Noun

In [39]:
# load vocab
with open('./vocab/vocab_index.pickle', 'rb') as f:
    vocab_index = pickle.load(f)
with open('./vocab/vocab_inverted_index.pickle', 'rb') as f:
    vocab_inverted_index = pickle.load(f)

In [42]:
def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])

def text_to_index(tokens):
    oov_id = 1
    indexes = []
    for tok in tokens:
        if tok in vocab_index:
            indexes.append(vocab_index[tok])
        else:
            indexes.append(oov_id)

    return indexes

In [45]:
pad_id = 0

x_variable = [text_to_index(_) for _ in morped]

sentence_size = 20
x_padded = sequence.pad_sequences(x_variable,
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

In [46]:
x_padded

array([[  51,  100,    1, ...,    1,  204,    1],
       [2191,    1,    0, ...,    0,    0,    0],
       [   6,    2,    0, ...,    0,    0,    0],
       ...,
       [ 191,   37, 1522, ...,    0,    0,    0],
       [  15,   38,    0, ...,    0,    0,    0],
       [  90,  241,  461, ...,    0,    0,    0]])

# Load Model 

In [8]:
new_model = keras.models.load_model('../model/cnn_oversample.h5')
new_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          50000     
_________________________________________________________________
conv1d (Conv1D)              (None, None, 32)          992       
_________________________________________________________________
dropout (Dropout)            (None, None, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          3104      
_________________________________________________________________
global_max_pooling1d (Globa

In [54]:
dump_data['predict'] = new_model.predict_classes(x_padded)

In [65]:
len(dump_data[dump_data.label == dump_data.predict]) /len(dump_data)

0.908

#### 전체 채팅 중 유해채팅 비율

In [68]:
dump_data.predict.sum() / len(dump_data)

0.146

> 5%가 적당하다